[Oregon Curriculum Network](http://4dsolutions.net/ocn/)<br/>
[School of Tomorrow](School_of_Tomorrow.ipynb)


# MAKING SHAPES

![ball_nest.gif](ball_nest.gif)

How do I make these animated GIFs such as the one above? As you might expect: there's a pipeline, a sequence of steps. 

First comes [a Python script](https://github.com/4dsolutions/School_of_Tomorrow/blob/master/flex_scripts3.py#L407) that, in turn, depends on several already-written modules, some by me, some by others. 

Then comes the [POV-Ray step](https://povray.org/), wherein I "render" (turn into a picture) what the Python script has put out: a file in Scene Description Language (file extension: pov). 

<a data-flickr-embed="true" href="https://www.flickr.com/photos/kirbyurner/55077921143/in/dateposted/" title="Povray on a MacPro"><img src="https://live.staticflickr.com/65535/55077921143_b3459f9381_z.jpg" width="640" height="393" alt="Povray on a MacPro"/></a><script async src="//embedr.flickr.com/assets/client-code.js" charset="utf-8"></script>

Finally, I've been using a 3rd party package named [Fiji](https://fiji.sc), a repackaging of ImageJ (written in Java) to crop and stitch the stills (png files) into a single GIF that loops through its two or more frames automatically.

The Python logic that computes the XYZ coordinates of vertex and edge objects, which in turn define polys (polyhedrons) resides mostly in the [flextegrity module](flextegrity.py). However, underlying `flextegrity` and also used directly, is the [qrays module](qrays.py), which implements `Qvector` type objects, or quadrays.

Finally, my logic depends on [sympy](https://www.sympy.org/en/index.html) to carryout algebraic manipulations and simplifications, with conversion to arbitrary precision decimal digits if and when these are called for, such as in the volumes table below. 

In [3]:
from flextegrity import pov_header, Cuboctahedron, Cube, Octahedron, RT
from flextegrity import Tetrahedron, InvTetrahedron, RD, PD, Icosahedron, Mite
from flextegrity import Edge, draw_edge, draw_poly, draw_vert, half, ORIGIN, PHI
from qrays import Qvector, Vector, A, B, C, D

import numpy as np
import sympy as sy
from sympy import sqrt as rt2, sin, cos
from mpmath import radians

from itertools import permutations
g = permutations((2,1,1,0))
UNIQUE = {p for p in g}  # set comprehension

IVM_DIRS = {Qvector(x) for x in UNIQUE}

Svol = (PHI **-5)/2  
Evol = (rt2(2)/8) * (PHI ** -3)

sfactor = Svol/Evol

CLOSEUP = \
"""
   
// perspective (default) camera
camera {
  location  <3, 0.1, 0.2>
  rotate    <35, 20, 33.0>
  look_at   <0.0, 0.0,  0.0>
  right     x*image_width/image_height
}

"""

That big import and constant defining going on above is how I, behaving like a typical Pythonista, usually start my various rendering projects. Python modules typically begin with such logic. 

In this Notebook format, I'm using code cells separated by MarkDown cells. However in it's native context wherein I'm not using text and graphics intensively, to explicate the workflow and logic, I would not use a notebook at all, just a standalone python program with the py file extension. Such a version of this notebook's code is linked above, where "Python script" in mentioned.

I know what tools I'm gonna want: quadrays, polyhedrons, trig functions and so on.

The code cell above brings in the polyhedra, the quadrays, plus defines some new constants, such as the volumes of the E and S modules in terms of PHI and 2nd root of 2. Their ratio, S to E, is called the sfactor and isn't used in this particular demonstration project. But lets keep it around anyway, in the name of realism.

These globals, including some that get imported, such as PHI, will get used further on (scripts run top to bottom).

In [10]:
PHI

1/2 + sqrt(5)/2

In [11]:
PHI.evalf(50)

1.6180339887498948482045868343656381177203091798058

In [14]:
IVM_DIRS  # the 12 quadrays to the surrounding 12-around-1 CCP balls, not needed in this demo

{ivm_vector(a=0, b=1, c=1, d=2),
 ivm_vector(a=0, b=1, c=2, d=1),
 ivm_vector(a=0, b=2, c=1, d=1),
 ivm_vector(a=1, b=0, c=1, d=2),
 ivm_vector(a=1, b=0, c=2, d=1),
 ivm_vector(a=1, b=1, c=0, d=2),
 ivm_vector(a=1, b=1, c=2, d=0),
 ivm_vector(a=1, b=2, c=0, d=1),
 ivm_vector(a=1, b=2, c=1, d=0),
 ivm_vector(a=2, b=0, c=1, d=1),
 ivm_vector(a=2, b=1, c=0, d=1),
 ivm_vector(a=2, b=1, c=1, d=0)}

In this Notebook, my plan is to make an animated GIF with the following frames:

* the Icosahedron, already defined, and a reference IVM ball, start the show
* the PD (Icosa's dual) gets added
* the RT (their sum total) appears (their "begot")
* the RT shrinks down to embrace the IVM ball more tightly
* show the RT shrink-wrapping the IVM ball with no other guys in view

Abbreviations Used:

PD:  pentagonal dodecahedron<br />
RT:  rhombic triacontahedron<br />
IVM: isotropic vector matrix (think of rods connecting adjacent balls in a CCP packing)<br />
CCP: cubic closest packing<br />

That's it for now. We could keep going with more frames.

I'm tempted to shrink that RT_E just a tad further, from volume ~5.0078 (120 E modules) to volume 5 exactly (that's the RT_T), then to expand it (volume-wise) by 5/2 to volume 7.5 exactly, where some of its corners would precisely align with the RD's of volume six. 

We put a K module here sometimes (120th of a 7.5 tetra volumed RT_K; each K module volume 1/16th, half the Mite's volume where Mite =  A- A+ B+ or B- A- A+).

But I'll save all that for another notebook (see Bonus Feature). 

Let's get our cast of characters on the scene then.

In [4]:
def test7():
    global ic, pd, rt, rt_e
    
    ic    = Icosahedron()    # edges D
    ic.edge_radius = 0.03

    pd  = PD()               # Icosa's dual
    pd.edge_radius = 0.03

    rt  = RT()               # their "begot" 
    rt.edge_radius = 0.03
    
    rt_e    = rt * (1/PHI)          
    rt_e.edge_radius = 0.03

test7()  # run the above function

That's enough code to instantiate our tiny cast of relevant polyhedra (others are still waiting in the wings, or behind the scenes, for other scenarios), minus the IVM ball itself (of diameter D and radius R) which we think of as an enlarged vertex that we'll add when it's time to actually generate our POV-Ray scene description language (coming up).

You may have noticed the multi-line string named CLOSEUP, controlling camera position: that's a fragment of scene description language. This text, along with other saved boilerplate, along with text generated at the time of computation (when the Python runs), makes up the pov files. The Python-computed parts open at the start of each `with` block below, with the name of the pov file it is about to generate.

Each `with` block is essentially saying: in the context of having this particular text file open, execute such functions as draw_poly and draw_vert to output our Python-defined geometrical objects in the form of scene description language that [POV-Ray](https://povray.org) will understand.

The code above is sufficient to give us an initial volumes table, but everything is still in Python. As of the code cell above, nothing has been written out for rendering as a ray-tracing (a picture) yet.

However before computing a volumes table, lets just check the icosahedron object by itself, as a special case example of a polyhedron. 

When a polyhedron is first instantiated by a line of code such as:

```python
    ic    = Icosahedron()    # edges D
    ic.edge_radius = 0.03
```

it enters the stage with a default size, color, and orientation. The thickness of its 30 edges (V + F = E + 2; 12 + 20 = 30 + 2) is an attribute we may wish to change, to thicker or thinner than the default. This is done by adjusting edge_radius, as shown. You might imagine 0.03 is in terms of millimeters (choose any concrete scale and/or medium for your imaginary construct and convey this by means of texture maybe, as we do with our IVM ball, by making it look made of stone, a greenish marble).

If the programmer wishes to resize a polyhedron, this is done with the multiplication operator, as we see in the line above:

```python
    rt_e    = rt * (1/PHI)
```

Multiplying the default rt of volume ~21.21 by `(1/PHI)` shrinks said rt in all linear dimensions by that amount, meaning volume shrinks as a 3rd power of same, yielding ~5.0078. 

The result of applying this linear scale factor is a new polyhedron with its dimensions adjusted accordingly. 

In this case, the default RT has now been copied and shrunk down, such that rt_e has center-to-diamond-face-centers of radius R (same as the IVM ball, which it encases).

The icosahedron enters the stage as D-edged, where D is the Diameter of our IVM ball. 

It's the same icosahedron described in most Jitterbug Transformation discussions, the one that derives from Jitterbugging the volume-20 cuboctahedron (if I may be permitted this archaic vocabulary).

In [5]:
ic.volume

5*sqrt(2)*(1/2 + sqrt(5)/2)**2

Thanks to `sympy` it volume is remembered in algebraic form, and renders under the hood using $LaTex$.

If you're practiced as recognizing PHI ($\phi$), you'll see it embedded in the above expression.

In [15]:
ic.simpler_volume = 5 * rt2(2) * PHI * PHI  # making up a new attribute just for this instance
ic.simpler_volume

5*sqrt(2)*(1/2 + sqrt(5)/2)**2

A string version, in standard Python syntax, may be requested instead.

In [4]:
str(ic.volume)

'5*sqrt(2)*(1/2 + sqrt(5)/2)**2'

Or which might ask for a decimal expansion; how many digits is up to us.

In [5]:
ic.volume.evalf(50)

18.512295868219161196009899292654531923571426913640

In [9]:
ic.simpler_volume.evalf(50)  # obviously the same, yet entered using PHI

18.512295868219161196009899292654531923571426913640

So now lets build that primitive volumes table. Each polyhedron knows its own volume internally.

In [6]:
print(
"""
Icosahedron: {0:>40}   {4:>12.11g}
PD:          {1:>40}   {5:>12.11g}
RT:          {2:>40}   {6:>12.11g}
RT_E:        {3:>40}   {7:>12.10g}
""".format(str(ic.volume), str(pd.volume), str(rt.volume), str(rt_e.volume), 
           ic.volume.evalf(), pd.volume.evalf(), rt.volume.evalf(), rt_e.volume.evalf()))


Icosahedron:           5*sqrt(2)*(1/2 + sqrt(5)/2)**2   18.512295868
PD:              3*sqrt(2)*(1 + (1/2 + sqrt(5)/2)**2)   15.350018208
RT:                                        15*sqrt(2)   21.213203436
RT_E:                 15*sqrt(2)/(1/2 + sqrt(5)/2)**3    5.007758031



Again, you will see PHI floating around, suggesting further simplifications

### The Five Frames

The screenshot below gives a sense of the POV-Ray windowed environment. 

The pov file appears to the left, with the R button setting off the rendering process, which, when running, is monitored in the window on the upper right (of course these windows maybe rearranged). 

The output window is centered behind the others.
<br />
<br />
<a data-flickr-embed="true" href="https://www.flickr.com/photos/kirbyurner/55077921148/in/photostream/" title="Screen Shot 2026-02-04 at 4.33.33 AM"><img src="https://live.staticflickr.com/65535/55077921148_8f9949586f_z.jpg" width="640" height="360" alt="Screen Shot 2026-02-04 at 4.33.33 AM"/></a><script async src="//embedr.flickr.com/assets/client-code.js" charset="utf-8"></script>
<br /><br />
The Icosahedron, already defined, and a reference IVM ball, start the show...

In [7]:
with open("genesis_1.pov", "w") as T:
    T.write(pov_header) 
    T.write(CLOSEUP)
    draw_poly(ic, T)
    draw_vert(ORIGIN, "T_Stone18", half, T, texture=True)

<a data-flickr-embed="true" href="https://www.flickr.com/photos/kirbyurner/55076692644/in/dateposted/" title="Frame 1: Icosa + IVM Ball"><img src="https://live.staticflickr.com/65535/55076692644_e9b2bde5fd.jpg" width="500" height="436" alt="Frame 1: Icosa + IVM Ball"/></a><script async src="//embedr.flickr.com/assets/client-code.js" charset="utf-8"></script>

The PD (Icosa's dual) gets added...

In [8]:
with open("genesis_2.pov", "w") as T:
    T.write(pov_header) 
    T.write(CLOSEUP)
    draw_poly(ic, T)
    draw_poly(pd, T)
    draw_vert(ORIGIN, "T_Stone18", half, T, texture=True)

<a data-flickr-embed="true" href="https://www.flickr.com/photos/kirbyurner/55076435251/in/dateposted/" title="Frame 2: Icosa + PD (dual)"><img src="https://live.staticflickr.com/65535/55076435251_1302f67720.jpg" width="500" height="436" alt="Frame 2: Icosa + PD (dual)"/></a><script async src="//embedr.flickr.com/assets/client-code.js" charset="utf-8"></script>

the RT (their sum total) appears (their "begot")...

In [9]:
with open("genesis_3.pov", "w") as T:
    T.write(pov_header) 
    T.write(CLOSEUP)
    draw_poly(ic, T)
    draw_poly(pd, T)
    draw_poly(rt, T)
    draw_vert(ORIGIN, "T_Stone18", half, T, texture=True)

<a data-flickr-embed="true" href="https://www.flickr.com/photos/kirbyurner/55076692634/in/dateposted/" title="Frame 3: Icosa + PD Beget RT"><img src="https://live.staticflickr.com/65535/55076692634_61ac44f5a0.jpg" width="500" height="436" alt="Frame 3: Icosa + PD Beget RT"/></a><script async src="//embedr.flickr.com/assets/client-code.js" charset="utf-8"></script>

The RT shrinks down to embrace the IVM ball more tightly...

In [10]:
with open("genesis_4.pov", "w") as T:
    T.write(pov_header) 
    T.write(CLOSEUP)
    draw_poly(ic, T)
    draw_poly(pd, T)
    draw_poly(rt_e, T)
    draw_vert(ORIGIN, "T_Stone18", half, T, texture=True)

<a data-flickr-embed="true" href="https://www.flickr.com/photos/kirbyurner/55076640078/in/dateposted/" title="Frame 4: RT Shrinks by 1/PHI In All Linear DImensions"><img src="https://live.staticflickr.com/65535/55076640078_6ae432e5d1.jpg" width="500" height="436" alt="Frame 4: RT Shrinks by 1/PHI In All Linear DImensions"/></a><script async src="//embedr.flickr.com/assets/client-code.js" charset="utf-8"></script>

Show the RT shrink-wrapping the IVM ball with no other guys in view

In [11]:
with open("genesis_5.pov", "w") as T:
    T.write(pov_header) 
    T.write(CLOSEUP)
    draw_poly(rt_e, T)
    draw_vert(ORIGIN, "T_Stone18", half, T, texture=True)

<a data-flickr-embed="true" href="https://www.flickr.com/photos/kirbyurner/55076442931/in/dateposted/" title="Frame 5: RT_E Hugs IVM Ball"><img src="https://live.staticflickr.com/65535/55076442931_f2f02d5786.jpg" width="500" height="436" alt="Frame 5: RT_E Hugs IVM Ball"/></a><script async src="//embedr.flickr.com/assets/client-code.js" charset="utf-8"></script>

With all five frames rendered individually, we're ready to stitch them together in a final product...

![genesis_story.gif](genesis_story.gif)

### Bonus Feature

<a data-flickr-embed="true" href="https://www.flickr.com/photos/kirbyurner/55076935791/in/dateposted/" title="Making RTs using Scale Factors"><img src="https://live.staticflickr.com/65535/55076935791_c4c1ecbf94.jpg" width="500" height="281" alt="Making RTs using Scale Factors"/></a><script async src="//embedr.flickr.com/assets/client-code.js" charset="utf-8"></script>


Without actually rendering a sixth frame, let's scale down our rt_e of volume ~5.0078 to exactly 5, through application of the tfactor. 

Note the tfactor is understood to be a linear scale factor, meaning when we apply it directly to our polyhedron, the volume is automatically scaled by a 3rd power of that.

The sfactor, mentioned above, is understood to scale by volume, turning the cuboctahedron of volume 2.5, for example, into the icosahedron within the volume 4 octahedron, through two multiplications thereof. 

One application of the sfactor turns the icosahedron of volume ~18.51 into a volume 20, the cuboctahedron.

In [16]:
tfactor = sy.Rational(2,3)**sy.Rational(1,3) * (PHI / rt2(2))
tfactor

2**(5/6)*3**(2/3)*(1/2 + sqrt(5)/2)/6

In [17]:
tfactor.evalf()

0.999483332262343

In [18]:
rt_t = rt_e * tfactor # linear scale factor

In [19]:
rt_t.volume  # the result is exact, thanks to sympy

5

Now that we have rt_t, we may scale it up by 3/2 volume-wise, a 3rd root of that length-wise, to get the rt_k of volume 7.5, made of 120 K modules, should we wish to expand BEAST into BASKET.

In [20]:
rt_k = rt_t * (sy.Rational(3,2) ** sy.Rational(1,3)) # 3/2 made linear

In [21]:
rt_k.volume

15/2

In [22]:
kmod = rt_k.volume/120
kmod

1/16

Let's add to our volumes table (above):

In [26]:
print(
"""
Shapes: In Order of Appearance

Icosahedron: {0:>40}   {6:>12.11g}
PD:          {1:>40}   {7:>12.11g}
RT:          {2:>40}   {8:>12.11g}
RT_E:        {3:>40}   {9:>12.10g}
RT_T:        {4:>40}   {10:>12.10g}
RT_K:        {5:>40}   {11:>12.10g}
""".format(str(ic.volume), str(pd.volume), str(rt.volume), str(rt_e.volume), str(rt_t.volume), str(rt_k.volume), 
           ic.volume.evalf(), pd.volume.evalf(), rt.volume.evalf(), rt_e.volume.evalf(), rt_t.volume.evalf(), rt_k.volume.evalf()))


Shapes: In Order of Appearance

Icosahedron:           5*sqrt(2)*(1/2 + sqrt(5)/2)**2   18.512295868
PD:              3*sqrt(2)*(1 + (1/2 + sqrt(5)/2)**2)   15.350018208
RT:                                        15*sqrt(2)   21.213203436
RT_E:                 15*sqrt(2)/(1/2 + sqrt(5)/2)**3    5.007758031
RT_T:                                               5    5.000000000
RT_K:                                            15/2    7.500000000



*Related Notebooks*:

[Generating the FCC](Flextegrity_Lattice.ipynb)<br/>
[Building a Volumes Table](CurricDevel.ipynb)<br />
[Vanes](Vanes.ipynb) (more about Quadrays)<br />
[Mapping Quadrays](Mapping_Quadrays.ipynb)<br />

*Related Slides*:

[The BASKET Modules: Synergetics Particle Zoo](https://docs.google.com/presentation/d/13QLfgKo6kyX0j0K0RJ7W0uisB0ZyeBiK-_qbzytipck/edit?usp=sharing)